In [1]:
import os

In [2]:
%pwd

'd:\\LaboratoireDeveloppement_TI\\IA\\Machine Learning\\deeplearningproject\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\LaboratoireDeveloppement_TI\\IA\\Machine Learning\\deeplearningproject\\Chicken-Disease-Classification-Project'

In [ ]:
import os
import sys
import time
import tensorflow as tf
from pathlib import Path
from src.constants import *
from zipfile import ZipFile
import urllib.request as request
from urllib.parse import urlparse
from dataclasses import dataclass
from src.logging.logger import logging
from src.exception.exception import CustomException
from src.utils.common import fn_read_yaml, fn_create_directories, fn_save_json

model = tf.keras.models.load_model("artifacts/training/model.h5")

@dataclass(frozen=True)
class _EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'artifacts/training/model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = fn_read_yaml(config_filepath)
        self.params = fn_read_yaml(params_filepath)      
        
        fn_create_directories([self.config.artifacts_root])
        
    def fn_get_validation_config(self) -> _EvaluationConfig:
        eval_config = _EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [ ]:
class Evaluation:
    def __init__(self, config: _EvaluationConfig):
        self.config = config
        
    def fn_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def fn_load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def fn_evaluation(self):
        self.model = self.fn_load_model(self.config.path_of_model)
        self.fn_valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def fn_save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        fn_save_json(path=Path("scores.json"), data=scores)

In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.fn_get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.fn_evaluation()
    evaluation.fn_save_score()
    
except Exception as e:
    raise CustomException(e, sys)